In [132]:
import numpy as np
from scipy.optimize import minimize, basinhopping
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.quantum_info import Operator

from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFT

In [227]:
params = ParameterVector('theta',6)
# this is a very bad universal 2q gate, but just a toy example
u = QuantumCircuit(2)
u.rx(params[0],0)
u.rx(params[1],1)
u.crx(params[2],0,1)
u.cry(params[3],1,0)
u.rz(params[4],0)
u.rz(params[5],1)

# target circuit
# v = QuantumCircuit(2)
# v.h(0)
# v.crx(np.pi,0,1)
# v.cry(0.1,1,0)
# V = Operator(v).data  # matrix representation of v
qft_circ = QFT(num_qubits=2, approximation_degree=0, do_swaps=False, inverse=False, insert_barriers=False, name=None)
v = qft_circ

In [228]:
state00 = Statevector.from_label('00')
state01 = Statevector.from_label('01')
state10 = Statevector.from_label('10')
state11 = Statevector.from_label('11')
input_states = [state00, state01, state10, state11]
outcome_states = [state.evolve(v) for state in input_states]
observables = [state.to_operator() for state in outcome_states]

In [229]:
for i in range(len(outcome_states)):
    print(outcome_states[i])

Statevector([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j],
            dims=(2, 2))
Statevector([ 5.000000e-01+0.j , -5.000000e-01+0.j ,  3.061617e-17+0.5j,
             -3.061617e-17-0.5j],
            dims=(2, 2))
Statevector([ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
            dims=(2, 2))
Statevector([ 5.000000e-01+0.j , -5.000000e-01+0.j , -3.061617e-17-0.5j,
              3.061617e-17+0.5j],
            dims=(2, 2))


In [239]:
type(observables[0])

qiskit.quantum_info.operators.operator.Operator

In [231]:
def fidelity_obj(x, *args):
    u, input_states, observables = args
    U = Operator(u.bind_parameters(x))
    output_states = [state.evolve(U) for state in input_states]
        
    fid = 0
    for idx, state in enumerate(output_states):
        #fid += state.data.conj() @ observables[idx].data @ state.data
        fid += np.trace(observables[idx].data @ state.to_operator().data)
    return np.real(-fid/len(output_states))

In [233]:
initial_guess = np.array([0,0,0,0,0,0])
result = minimize(fidelity_obj, initial_guess, args=(u, input_states, observables), method='cobyla')
#minimizer_kwargs = dict(method="L-BFGS-B", args=(u, input_states, observables))
#result = basinhopping(fidelity_obj, initial_guess, minimizer_kwargs=minimizer_kwargs)
print(result)
print('Final circuit')
print(u.bind_parameters(result.x).draw())

U = Operator(u.bind_parameters(result.x))
output_states = [state.evolve(U) for state in input_states]
print(output_states)

     fun: -0.9999999977066174
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 145
  status: 1
 success: True
       x: array([-1.57080431e+00, -1.57077166e+00, -9.59529295e-05,  1.57077012e+00,
       -1.57075940e+00, -7.85464759e-01])
Final circuit
     ┌───────────────────────┐                            »
q_0: ┤ Rx(-1.57080430779352) ├─────────────■──────────────»
     ├───────────────────────┤┌────────────┴─────────────┐»
q_1: ┤ Rx(-1.57077165841904) ├┤ Rx(-9.59529294948062e-5) ├»
     └───────────────────────┘└──────────────────────────┘»
«     ┌──────────────────────┐┌───────────────────────┐ 
«q_0: ┤ Ry(1.57077012155327) ├┤ Rz(-1.57075940117197) ├─
«     └──────────┬───────────┘├───────────────────────┴┐
«q_1: ───────────■────────────┤ Rz(-0.785464758757777) ├
«                             └────────────────────────┘
[Statevector([0.19133646+0.46194646j, 0.19131175+0.46193505j,
             0.19137473+0.4619324j , 0.19133182+0.46195017j],
            dim

In [235]:
outcome_states

[Statevector([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j],
             dims=(2, 2)),
 Statevector([ 5.000000e-01+0.j , -5.000000e-01+0.j ,  3.061617e-17+0.5j,
              -3.061617e-17-0.5j],
             dims=(2, 2)),
 Statevector([ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
             dims=(2, 2)),
 Statevector([ 5.000000e-01+0.j , -5.000000e-01+0.j , -3.061617e-17-0.5j,
               3.061617e-17+0.5j],
             dims=(2, 2))]

In [182]:
import numpy as np
from qiskit.algorithms.optimizers import QNSPSA
from qiskit.circuit.library import PauliTwoDesign
from qiskit.opflow import Z, StateFn

ansatz = PauliTwoDesign(2, reps=1, seed=2)
observable = Z ^ Z
initial_point = np.random.random(ansatz.num_parameters)

def loss(x):
    bound = ansatz.bind_parameters(x)
    return np.real((StateFn(observable, is_measurement=True) @ StateFn(bound)).eval())

fidelity = QNSPSA.get_fidelity(ansatz)
qnspsa = QNSPSA(fidelity, maxiter=300)
result = qnspsa.optimize(ansatz.num_parameters, loss, initial_point=initial_point)

In [224]:
def fidelity_obj(x):
    U = Operator(u.bind_parameters(x))
    output_states = [state.evolve(U) for state in input_states]
        
    fid = 0
    for idx, state in enumerate(output_states):
        #fid += state.data.conj() @ observables[idx].data @ state.data
        fid += np.trace(observables[idx].data @ state.to_operator().data)
    return np.real(-fid/len(output_states))

initial_point = np.random.random(u.num_parameters)
fidelity = QNSPSA.get_fidelity(u)
qnspsa = QNSPSA(fidelity, maxiter=300)
result = qnspsa.optimize(u.num_parameters, fidelity_obj, initial_point=initial_point)
print(result)

U = Operator(u.bind_parameters(result[0]))
output_states = [state.evolve(U) for state in input_states]
print(output_states)

(array([ 1.01255428, -3.69199167,  1.63194904,  2.19386059,  0.81007444,
        1.06203548]), -0.71917356615196, 2101)
[Statevector([ 0.07972573+0.22387888j,  0.20681972+0.17805448j,
             -0.15362723+0.4999597j , -0.19923612+0.74551204j],
            dims=(2, 2)), Statevector([ 0.12413663-0.04420641j, -0.32111908+0.37299683j,
              0.74251671+0.1974304j , -0.378915  -0.080656j  ],
            dims=(2, 2)), Statevector([ 0.79287326-0.28235087j,  0.39468044-0.07001132j,
             -0.11329023+0.05323042j, -0.25797915-0.22075856j],
            dims=(2, 2)), Statevector([-0.15655824-0.4396333j ,  0.12626456+0.71180133j,
             -0.2299808 +0.25994196j,  0.3194507 -0.19260962j],
            dims=(2, 2))]


In [226]:
outcome_states

[Statevector([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j],
             dims=(2, 2)),
 Statevector([ 5.000000e-01+0.j , -5.000000e-01+0.j ,  3.061617e-17+0.5j,
              -3.061617e-17-0.5j],
             dims=(2, 2)),
 Statevector([ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
             dims=(2, 2)),
 Statevector([ 5.000000e-01+0.j , -5.000000e-01+0.j , -3.061617e-17-0.5j,
               3.061617e-17+0.5j],
             dims=(2, 2))]

In [221]:
from qiskit.algorithms.optimizers import AQGD

def fidelity_obj(x):
    print(x)
    U = Operator(u.bind_parameters(x))
    output_states = [state.evolve(U) for state in input_states]
        
    fid = 0
    for idx, state in enumerate(output_states):
        #fid += state.data.conj() @ observables[idx].data @ state.data
        fid += np.trace(observables[idx].data @ state.to_operator().data)
    print(np.real(-fid/len(output_states)))
    return np.real(-fid/len(output_states))

initial_point = np.random.random(u.num_parameters)
aqgd = AQGD(maxiter=300)
result = aqgd.optimize(u.num_parameters, fidelity_obj, initial_point=initial_point)
print(result)

[ 0.62224882  0.5590185   0.14563938  0.20978552  0.52075515  0.85671525
  2.19304514  0.5590185   0.14563938  0.20978552  0.52075515  0.85671525
  0.62224882  2.12981483  0.14563938  0.20978552  0.52075515  0.85671525
  0.62224882  0.5590185   1.71643571  0.20978552  0.52075515  0.85671525
  0.62224882  0.5590185   0.14563938  1.78058185  0.52075515  0.85671525
  0.62224882  0.5590185   0.14563938  0.20978552  2.09155148  0.85671525
  0.62224882  0.5590185   0.14563938  0.20978552  0.52075515  2.42751158
 -0.94854751  0.5590185   0.14563938  0.20978552  0.52075515  0.85671525
  0.62224882 -1.01177782  0.14563938  0.20978552  0.52075515  0.85671525
  0.62224882  0.5590185  -1.42515694  0.20978552  0.52075515  0.85671525
  0.62224882  0.5590185   0.14563938 -1.3610108   0.52075515  0.85671525
  0.62224882  0.5590185   0.14563938  0.20978552 -1.05004118  0.85671525
  0.62224882  0.5590185   0.14563938  0.20978552  0.52075515 -0.71408107]


ValueError: Mismatching number of values and parameters. For partial binding please pass a dictionary of {parameter: value} pairs.